In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.client import device_lib
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
import numpy as np

In [28]:
def readDatasets(path):
    f = open(path, 'r', encoding='utf-8')
    texts = f.readlines()
    f.close()
    return texts

text_train = readDatasets('HerMajestySpeechesDataset/train.txt')
text_test = readDatasets('HerMajestySpeechesDataset/test.txt')
text_val = readDatasets('HerMajestySpeechesDataset/dev.txt')



In [29]:
tokenizer_train = Tokenizer(oov_token='<unk>', num_words = 2000)
tokenizer_train.fit_on_texts(text_train) 
texts2ids_train = tokenizer_train.texts_to_sequences(text_train)

texts2ids_test = tokenizer_train.texts_to_sequences(text_test)

texts2ids_val = tokenizer_train.texts_to_sequences(text_val)

In [30]:
np.max(np.concatenate(texts2ids_train))

1999.0

In [31]:
print(len(texts2ids_train[0]), len(texts2ids_test[0]), len(texts2ids_val[0]))

12 8 13


In [32]:
for i in texts2ids_train:
    i.append(2000)

for i in texts2ids_test:
    i.append(2000)

for i in texts2ids_val:
    i.append(2000)

In [33]:
print(len(texts2ids_train[0]), len(texts2ids_test[0]), len(texts2ids_val[0]))


13 9 14


In [34]:
def get_ngrams(frase, size=2):
    frase = np.concatenate((np.zeros(size), frase))

    ngrams_list = []
    for i in range(len(frase)-size):
        ngrams_list.append((tuple(frase[i:i+size]),frase[i+size]))
    return ngrams_list

all_ngrams = []
for i in texts2ids_train:
    all_ngrams += get_ngrams(i)


In [35]:
def co_table(lista_ocurrencias):
    table = {}
    for i in lista_ocurrencias:
        if i[0] in table:
            if i[1] in table[i[0]]:
                table[i[0]][i[1]] += 1
            else:
                table[i[0]][i[1]] = 1
        else:
            table[i[0]] = {}
            table[i[0]][i[1]] = 1
    return table

table = co_table(all_ngrams)

In [36]:
def generate_toyLM_ngram_a(table, context='aleatorio', n=15):
    if context == 'aleatorio':
        tmp = list(table.keys())
        context = tmp[np.random.randint(len(tmp))]

    cadena = list(context)
    for _ in range(n-len(context)):
        context = tuple(cadena[-len(context):])
        if context not in table.keys():
            break
        else:
            new = max(table[context], key=table[context].get)
            if new == 2000:
                break
            cadena.append(new)
    return tokenizer_train.sequences_to_texts([cadena])

generate_toyLM_ngram_a(table)

['sacrifice of their <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>']

In [37]:
def perplexity_ngrams(frases):
    tmp = []
    for frase in frases:
        n_grams = get_ngrams(frase)
        perplexity = 1
        for context, following in n_grams:
            if context in table.keys():
                denominador = sum(table[context].values())
                if following in table[context].keys():
                    numerador = table[context][following]
                else:
                    numerador = 0
            else:
                denominador, numerador = 0, 0

            numerador += 1
            denominador += 2000

            perplexity *= 1/(numerador/denominador)
        tmp.append(perplexity**(1/len(n_grams)))

    return np.mean(tmp)

In [38]:
perplexity_ngrams(texts2ids_test)

992.1306822885472

In [39]:
def generate_toyLM_ngram_b(table, context='aleatorio', n=15):
    if context == 'aleatorio':
        tmp = list(table.keys())
        context = tmp[np.random.randint(len(tmp))]

    cadena = list(context)
    for i in range(n-len(context)):
        context = tuple(cadena[-len(context):])
        if context not in table.keys():
            break
        else:
            lista_tmp = list()
            for i in table[context]:
                for j in range(table[context][i]):
                    lista_tmp.append(i)
            new = np.random.randint(len(lista_tmp))
            if new == 2000:
                break
            cadena.append(lista_tmp[new])
    return tokenizer_train.sequences_to_texts([cadena])

generate_toyLM_ngram_b(table)


['all important popular level that so many of them <unk> over the many organisations with']

# toyLM_LSTM

In [40]:
def train_generate(text, max_seq_length=10):
    train_set = dict()
    for frase in text:
        for word_index in range(len(frase)):
            if word_index < max_seq_length:
                train_set[tuple(pad_sequences([frase[:word_index]], maxlen=max_seq_length)[0])] = frase[word_index]
    return train_set
    
train_set = train_generate(texts2ids_train)
test_set = train_generate(texts2ids_test)
val_set = train_generate(texts2ids_val)

In [41]:
def train_generate(text, size=2):
    x = []
    y = []
    for phrase in text:
        for context, following in get_ngrams(phrase,size):
            x.append(list(context))
            y.append(following)
    return np.array(x), to_categorical(np.array(y))

In [54]:
x_train, y_train = train_generate(texts2ids_train, 2)
x_test, y_test = train_generate(texts2ids_test, 2)
x_val, y_val = train_generate(texts2ids_val, 2)

print(x_val.shape, y_val.shape)

(12971, 2) (12971, 2001)


In [55]:
model = Sequential([
    Embedding(2001, 20),
    LSTM(64),
    Dense(2001, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=64, epochs=10)

Epoch 1/10
861/861 [==============================] - 6s 4ms/step - loss: 5.8795 - accuracy: 0.0842   

In [84]:
def perplexity_lstm(model, frases):
    tmp = []
    for frase in frases:
        x_test, _ = train_generate([frase])
        perplexity = 1
        prob = model.predict(x_test)
        for num, i in enumerate(prob):
            perplexity *= 1/i[frase[num]]
        tmp.append(perplexity**(1/len(x_test)))
    return np.mean(tmp)

perplexity_lstm(model, texts2ids_test)

126.27498186478542

In [79]:
print(len(texts2ids_test))

491


In [82]:
def perplexity_lstm(model, frases):
    tmp = list()
    ct=0
    for frase in frases:
        ct+=1
        print(ct)
        n_grams = get_ngrams(frase)
        perplexity = 1
        for context, following in n_grams:
            prob = model.predict(np.array([context]))

            perplexity *= 1/prob[0][int(following)]
        tmp.append(perplexity**(1/len(n_grams)))
    return np.mean(tmp)
perplexity_lstm(model, texts2ids_test)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


126.27498219683494

In [ ]:
def generate_toyLM_lstm_a(model, context='aleatorio', n=15):
    if context=='aleatorio':
        context = list(np.random.randint(0, 5616,2))

    cadena = context
    for _ in range(n-len(context)):
        context = cadena[-len(context):]
        new = np.argmax(model.predict(np.array([context])))
        if new == 5615:
            break
        cadena.append(new)
    return tokenizer_train.sequences_to_texts([cadena])

generate_toyLM_lstm_a(model)

In [ ]:
def generate_toyLM_lstm_b(model, context='aleatorio', n=15):
    if context=='aleatorio':
        context = list(np.random.randint(0, 2001,2))

    cadena = context
    for _ in range(n-len(context)):
        context = cadena[-len(context):]
        probs = model.predict(np.array([context]))
        new = np.random.choice(range(2001), p=probs[0])
        if new == 5615:
            break
        cadena.append(new)
    return tokenizer_train.sequences_to_texts([cadena])

generate_toyLM_lstm_b(model)
